# Doc2Vecを用いたテキスト分類

このノートブックでは、Doc2Vecを用いたテキスト分類の方法を説明します。データセットとしては、Kaggleで公開されている「[Sentiment and Emotion in Text](https://www.kaggle.com/c/sa-emotions/data)」を利用します。

このデータセットは、一般的なタスクであるセンチメント分析の1つで、テキストの感情的内容（喜び、悲しみ、怒りなど）のラベルを含んでいます。13個のラベルに数百から数千の例が含まれています。このデータのサブセットは、MicrosoftのCortana Intelligence Galleryにアップロードした実験で使用されています．

## 準備
### パッケージのインポート

In [1]:
import nltk
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### データセットのダウンロード

In [2]:
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
!ls -lah DATAPATH

--2021-09-07 11:20:36--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2479133 (2.4M) [text/plain]
Saving to: ‘DATAPATH/train_data.csv’

train_data.csv      100%[===================>]   2.36M  --.-KB/s    in 0.08s   

2021-09-07 11:20:36 (27.8 MB/s) - ‘DATAPATH/train_data.csv’ saved [2479133/2479133]

--2021-09-07 11:20:36--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuser

### データセットの読み込み

In [3]:
filepath = "DATAPATH/train_data.csv"
df = pd.read_csv(filepath)
print(df.shape)
df.head()

(30000, 2)


,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
df['sentiment'].value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [5]:
# 3カテゴリだけ残す
shortlist = ['neutral', "happiness", "worry"]
df_subset = df[df['sentiment'].isin(shortlist)]
df_subset.shape

(16759, 2)

## テキストの前処理

Tweetでは以下のことを考慮する必要があります。

- @mentionやURLは除去するか
- 通常のトークナイザーではなくTweet用のトークナイザーを使うか
- ストップワードや数字はどうするか

In [6]:
# strip_handlesはTwitterのハンドル名のような個人情報を除去します。
# そのような情報はTweetの感情を分類するのにあまり貢献しないと考えられます。
# preserve_case=Falseは、小文字に変換するための引数です。
tweeter = TweetTokenizer(strip_handles=True, preserve_case=False)
mystopwords = set(stopwords.words("english"))

# Tweetをトークナイズし、ストップワードや数字を除去するための関数です。
# 句読点と感情を表す顔文字などは、タスクに関連するので残しておきます。
def preprocess_corpus(texts):
    def remove_stops_digits(tokens):
        # 入れ子になった関数。ストップワードと数字をトークンのリストから除去
        return [token for token in tokens if token not in mystopwords and not token.isdigit()]
    # 上記で定義した関数を使って、Twitterトークナイザーの出力をさらに処理
    return [remove_stops_digits(tweeter.tokenize(content)) for content in texts]

# df_subsetは3つのカテゴリのみを含む
mydata = preprocess_corpus(df_subset['content'])
mycats = df_subset['sentiment']
print(len(mydata), len(mycats))

16759 16759


## Doc2Vecの学習

In [7]:
#　データを学習用とテスト用に分割
train_data, test_data, train_cats, test_cats = train_test_split(
    mydata,
    mycats,
    random_state=1234
)

# doc2vec形式の学習データを準備
train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(train_data)]

# Tweetの表現を学習するために、doc2vecモデルを学習
model = Doc2Vec(vector_size=50, alpha=0.025, min_count=5, dm=1, epochs=100)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")
print("Model Saved")

Model Saved


## 分類モデルの学習

In [8]:
# 学習したモデルを用いて、学習データとテストデータに対する特徴表現を推論
model= Doc2Vec.load("d2v.model")

# 安定した表現を得るために、複数ステップの推論をする
train_vectors =  [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in train_data]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in test_data]

# ロジスティック回帰のような分類機を使う
from sklearn.linear_model import LogisticRegression

# クラスは不均衡なので、class_weight="balanced"を設定
myclass = LogisticRegression(class_weight="balanced")
myclass.fit(train_vectors, train_cats)

preds = myclass.predict(test_vectors)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_cats, preds))

              precision    recall  f1-score   support

   happiness       0.33      0.52      0.40       713
     neutral       0.46      0.53      0.49      1595
       worry       0.60      0.39      0.47      1882

    accuracy                           0.47      4190
   macro avg       0.46      0.48      0.46      4190
weighted avg       0.50      0.47      0.47      4190

